In [ ]:
# Softmax classifier for guessing minesweeper board position and whether it has a mine or not

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt
import multiprocessing as mp

/home/ruben/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (8,8)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# Clears a square on the minesweeper board.
# If it had a mine, return true
# Otherwise if it has no adjacent mines, recursively run on adjacent squares
# Return false
def clearSquare(board,adjacency,row,col):
    rows,cols = dimensions
    if board[row,col] == 1:
        return True
    if adjacency[row,col] >= 0:
        return False
    n = 0
    for r in range(row-1,row+2):
        for c in range(col-1,col+2):
            if 0 <= r and r < rows and 0 <= c and c < cols:
                n += board[r,c]
    adjacency[row,col] = n
    if n == 0:
        for r in range(row-1,row+2):
            for c in range(col-1,col+2):
                if 0 <= r and r < rows and 0 <= c and c < cols:
                    clearSquare(board,adjacency,r,c)
    return False

In [4]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    clearProbability = r.uniform(0.05,0.5)
    result = np.full(dimensions,-1)
    for index, x in np.random.permutation(list(np.ndenumerate(board))):
        row,col = index
        if not(x) and result[row,col] == -1 and r.uniform(0,1) < clearProbability:
            clearSquare(board,result,row,col)
    return result

In [5]:
# Generates a random training batch of size n
def randomBoard(i):
    return(np.random.random(dimensions) < mineProbability)

def encodeCountsOneHot(counts):
    countsOneHot = np.zeros((counts.size,10))
    countsOneHot[np.arange(counts.size), counts.flatten() + 1] = 1
    return(countsOneHot.flatten())

def validGuesses(boardAndCounts):
    board,counts = boardAndCounts
    validGuesses = np.append(((counts == -1).astype(int) - board).flatten().astype(float),
        board.flatten().astype(float))
    validGuessesSum = sum(validGuesses)
    if validGuessesSum > 0:
        return(validGuesses / validGuessesSum)
    else:
        return(np.zeros(board.size*2))

def next_training_batch(n):
    try:
        cpus = mp.cpu_count()
    except NotImplementedError:
        cpus = 2   # arbitrary default
    
    pool = mp.Pool(processes=cpus)
    boards = pool.map(randomBoard, range(n))
    counts = pool.map(boardPartialMineCounts, boards)
    batch_xs = pool.map(encodeCountsOneHot, counts)
    batch_ys = pool.map(validGuesses, zip(boards,counts))
    return(batch_xs, batch_ys, boards)

In [6]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCountsOneHot = tf.placeholder(tf.float32, [None, size*10], name="mineCountsOneHot")
W = tf.Variable(tf.truncated_normal([size*10, size], stddev=0.01), name="W")
b = tf.Variable(tf.truncated_normal([size], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.01)
  return tf.Variable(initial)

In [8]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [9]:
y_image = tf.reshape(y, [-1, rows, cols, 1])

# First convolution layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(y_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second convolution layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Fully connected layer
W_fc1 = weight_variable([(rows // 4) * (cols // 4) * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, (rows // 4) * (cols // 4) * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output layer
W_fc2 = weight_variable([1024, size*2])
b_fc2 = bias_variable([size*2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [10]:
validGuessAverages = tf.placeholder(tf.float32, [None, size*2], name="validGuessAverages")

In [11]:
# Loss function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=validGuessAverages, logits=y_conv))

In [12]:
# Summaries for tensorboard
with tf.name_scope('W_reshape'):
    image_shaped_W = tf.reshape(W, [-1, size*10, size, 1])
    tf.summary.image('W', image_shaped_W, 1000)

with tf.name_scope('b_reshape'):
    image_shaped_b = tf.reshape(b, [-1, rows, cols, 1])
    tf.summary.image('b', image_shaped_b, 1000)

with tf.name_scope('W_conv1_reshape'):
    image_shaped_W_conv1 = tf.reshape(W_conv1, [-1, 5*5, 32, 1])
    tf.summary.image('W_conv1', image_shaped_W_conv1, 1000)

with tf.name_scope('W_conv2_reshape'):
    image_shaped_W_conv2 = tf.reshape(W_conv2, [-1, 5*32, 5*64, 1])
    tf.summary.image('W_conv2', image_shaped_W_conv2, 1000)

with tf.name_scope('W_fc1_reshape'):
    image_shaped_W_fc1 = tf.reshape(W_fc1, [-1, (rows // 4) * (cols // 4) * 64, 1024, 1])
    tf.summary.image('W_fc1', image_shaped_W_fc1, 1000)

with tf.name_scope('b_fc1_reshape'):
    image_shaped_b_fc1 = tf.reshape(b_fc1, [-1, 32, 32, 1])
    tf.summary.image('b_fc1', image_shaped_b_fc1, 1000)

with tf.name_scope('W_fc2_reshape'):
    image_shaped_W_fc2 = tf.reshape(W_fc2, [-1, 1024, size*2, 1])
    tf.summary.image('W_fc2', image_shaped_W_fc2, 1000)

with tf.name_scope('b_fc2_reshape'):
    image_shaped_b_fc2 = tf.reshape(b_fc2, [-1, rows*2, cols, 1])
    tf.summary.image('b_fc2', image_shaped_b_fc2, 1000)

_ = tf.summary.scalar('loss', cross_entropy)

In [13]:
# Optimiser
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [14]:
savePathPrefix = './tf.Mines7/'

modelDataPath = savePathPrefix + str(dimensions) + '/'

def summaryPath(run):
    return(savePathPrefix + 'runs/' + str(run) + '/')

In [15]:
# Create session and initialise or restore stuff
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(summaryPath(2), sess.graph)

In [16]:
tf.global_variables_initializer().run()

In [17]:
# Restore model?
saver.restore(sess, modelDataPath + "model-500")

INFO:tensorflow:Restoring parameters from ./tf.Mines7/(8, 8)/model-500


In [ ]:
# Train
for iteration in range(500,100001):
    batch_xs, batch_ys, _ = next_training_batch(10000)
    summary, loss, _ = sess.run([merged, cross_entropy, train_step],
                                 feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys, keep_prob: 0.5})
    writer.add_summary(summary, iteration)
    print('%s: Loss at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    if iteration % 10 == 0:
        save_path = saver.save(sess, modelDataPath + 'model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-07 22:27:56: Loss at step 500: 4.16753
Model saved in file: ./tf.Mines7/(8, 8)/model-500
2017-11-07 22:27:59: Loss at step 501: 4.16508
2017-11-07 22:28:01: Loss at step 502: 4.16522
2017-11-07 22:28:03: Loss at step 503: 4.16439
2017-11-07 22:28:06: Loss at step 504: 4.16712
2017-11-07 22:28:08: Loss at step 505: 4.16418
2017-11-07 22:28:10: Loss at step 506: 4.15744
2017-11-07 22:28:13: Loss at step 507: 4.15824
2017-11-07 22:28:16: Loss at step 508: 4.15448
2017-11-07 22:28:18: Loss at step 509: 4.15706
2017-11-07 22:28:21: Loss at step 510: 4.15516
Model saved in file: ./tf.Mines7/(8, 8)/model-510
2017-11-07 22:28:23: Loss at step 511: 4.15855
2017-11-07 22:28:26: Loss at step 512: 4.15605
2017-11-07 22:28:29: Loss at step 513: 4.15382
2017-11-07 22:28:32: Loss at step 514: 4.15822
2017-11-07 22:28:35: Loss at step 515: 4.15429
2017-11-07 22:28:37: Loss at step 516: 4.15056
2017-11-07 22:28:40: Loss at step 517: 4.15017
2017-11-07 22:28:42: Loss at step 518: 4.15399
2017-11

In [ ]:
# Test trained model on larger batch size
batch_xs, batch_ys, _ = next_training_batch(10000)
print(sess.run(cross_entropy, feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys}))

In [ ]:
# Run a test
batchSize = 10000
batch_xs, batch_ys,_ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
frees = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(frees))

In [ ]:
# Find boards that we failed on
batchSize = 1000
batch_xs, batch_ys, _ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
guesses = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
for i in range(batchSize):
    if guesses[i] == 1:
        print(batch_xs[i].reshape(dimensions))
        summary = sess.run(tf.summary.image('mine_miss', tf.reshape((batch_xs[i]+1).astype(float),[-1,rows,cols,1]), 100))
        writer.add_summary(summary)

In [ ]:
#batch_xs = [[-1,1,-1,0,0,0,-1,-1,-1,1,1,1,-1,-1,1,2,2,1,1,-1,2,1,1,-1,-1,2,2,-1,-1,2,-1,1,1,0,-1,-1,2,-1,-1,4,-1,2,-1,1,2,-1,1,0,1,2,-1,3,2,2,1,-1,2,-1,1,0,0,1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,0,0,3,-1,4,1,2,-1,1,-1,-1,0,0,0,2,-1,-1,-1,2,-1,1,0,0,0,-1,1,2,-1,2,1,2,2,3,3,2,-1,-1,1,-1,1,-1,0,1,2,-1,-1,-1,1,1,1,-1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,1,-1,-1,-1]]
batch_xs0 = [-1] * (size)
batch_xs0[0] = 1
batch_xs0[1] = 1
batch_xs0[cols] = 1

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: [batch_xs0]})
bestSquares = [pred.argmax() for pred in predictions]

print(bestSquares[0] // cols, bestSquares[0] % cols)

In [ ]:
np.save("./W", sess.run(W))

In [ ]:
np.save("./b", sess.run(b))

In [ ]:
np.savez("./model", sess.run([W,b]))